In [ ]:
from __future__ import division
import datetime
import time
import os
import sys

import numpy as np
from scipy import linalg
from matplotlib import rcParams
import matplotlib.pyplot as plt

sys.path.append('../')
from alg_tools_1d import build_G_fourier, dirac_recon_irreg_fourier, distance

In [ ]:
'''utility functions used for plotting'''
def plot_dirac(tk, ak, color='red', marker='*', ax=None, label=''):
    if ax is None:
        fig = plt.figure()
        ax = plt.gca()
    markerline211_1, stemlines211_1, baseline211_1 = \
        ax.stem(tk, ak, label=label)
    plt.setp(stemlines211_1, linewidth=1.5, color=color)
    plt.setp(markerline211_1, marker=marker, linewidth=1.5, markersize=8,
             markerfacecolor=color, mec=color)
    plt.setp(baseline211_1, linewidth=0)
    plt.xlim([-TAU/2, TAU/2])
    
def plot_diracs(ax1, tk, ak, tk_recon, ak_recon, title):
    c1 = [0, 0.447, 0.741]
    m1 ='^'
    plot_dirac(tk, ak, c1, m1, ax=ax1, label='Original Diracs')
    
    c2 = [0.850, 0.325, 0.098]
    m2 = '*'
    plot_dirac(tk_recon, ak_recon, c2, m2, ax=ax1, label='Estimated Diracs')

    plt.axhline(0, color='k')
    plt.ylim([1.17 * np.min(np.concatenate((ak, ak_recon, np.array(0)[np.newaxis]))),
              1.17 * np.max(np.concatenate((ak, ak_recon, np.array(0)[np.newaxis])))])
    plt.ylabel('amplitudes', fontsize=12)
    ax1.yaxis.set_label_coords(-0.095, 0.5)
    plt.legend(numpoints=1, loc=0, fontsize=9, framealpha=0.3,
               handletextpad=.2, columnspacing=0.6, labelspacing=0.05, ncol=2)
    plt.title(title, fontsize=12)

# Overview 

This notebook explains how to reconstruct a signal consisting of a K Diracs at unknown locations from Fourier-domain samples.  

## 1. Generate signal

We generate the FRI signal which we will then try reconstruct:

<br><center>
$ \displaystyle x = \sum_{k=1}^{K}  \alpha_k \delta(t - t_k) $ (1) 
</center>

*CODE: Inspect the signal and make sure you understand its parameters.*

In [ ]:
np.random.seed(7)

K = 5  # number of Diracs
TAU = 1  # period of the Dirac stream

# amplitudes of the Diracs
ak = np.sign(np.random.randn(K)) * (1 + (np.random.rand(K) - 0.5) / 1.)

# locations of the Diracs
tk = np.random.rand(K) * TAU - TAU / 2

# plot the signal.
plot_dirac(tk, ak)

## 2. Simulate measurements

We also simulate measurements by sampling from the Fourier transform of the above signal at randomly chosen locations $\omega_\ell$. 

$$y_{\ell} = \sum_{k=1}^K \alpha_k e^{-j \omega_\ell t_k}$$

for $\ell=1,\cdots,L$. 

*CODE: Do you understand the values of $M$, $B$ and $L$ in the code snippet below?*

In [ ]:
np.random.seed(3)

M = 21 # period of the spectrom; M * tau must be an odd number.
B = (2. * M + 1.) / TAU  # bandwidth of the sampling filter
L = 2 * M # number of Fourier domain measurements.

omega_ell = np.pi * (np.random.rand(L) * (2 * M - 1) - M)

# ground truth signal
tk_grid, omega_grid = np.meshgrid(tk, omega_ell)
y_ell_samp = np.dot(np.exp(-1j * omega_grid * tk_grid), ak)

# continuous signal (for plotting only)
omega_continuous = np.linspace(-np.pi * M, np.pi * M, num=1000)
tk_grid, omega_grid = np.meshgrid(tk, omega_continuous)
y_ell_continuous = np.dot(np.exp(-1j * omega_grid * tk_grid), ak)

## generate noisy signal
sigma_noise = 1e-1
#sigma_noise = 0 
noise = np.random.normal(scale=sigma_noise, loc=0, size=y_ell_samp.shape)
y_ell = y_ell_samp + noise
y_ell.imag = y_ell_samp.imag + noise


# plot the signal
plt.figure()
plt.plot(omega_continuous, np.real(y_ell_continuous), 'grey', label='continuous')
plt.plot(omega_ell, np.real(y_ell_samp), 'r*', label='samples')
plt.plot(omega_ell, np.real(y_ell), 'g*', label='noisy samples')
plt.xlabel('$\omega$')
plt.ylabel('Real')
plt.legend()

plt.figure()
plt.plot(omega_continuous, np.imag(y_ell_continuous), 'grey', label='continuous')
plt.plot(omega_ell, np.imag(y_ell_samp), 'r*', label='samples')
plt.plot(omega_ell, np.imag(y_ell), 'g*', label='noisy samples')
plt.xlabel('$\omega$')
plt.ylabel('Imaginary')
plt.legend()

## 3. Find standard form

Since the signal it is FRI, we know that we can find a signal of the standard form with a 1-to-1 relation to the original signal:

<center>
$ \displaystyle\hat{x}_m = \sum_{k=1}^{K} \beta_k u_k^m $ (2) 
</center>

*PEN AND PAPER: Find values of $\beta_k$ and $u_k$*. 

Since the above holds, we know that the signal can be anihilated by a filter $h$. 

*OPTIONAL: Show that for this simple example, this filter is given by*

$$ H(z) = h_0 \prod_{k=1}^K (1 - e^{-j\frac{2 \pi}{\tau} t_k} z^{-1}) $$

In [ ]:
def get_standard_form(ak, tk):
    ''' 
    :param ak: vector of Dirac amplitudes
    :param tk: vector of Dirac locations

    :return: vector of standard form coefficients
    '''
    ms = np.arange(-np.floor(B * TAU / 2.), 1 + np.floor(B * TAU / 2.))
    tk_grid, m_grid_gt = np.meshgrid(tk, ms)
    
    x_hat = 1. / TAU * np.dot(np.exp(-2j * np.pi / TAU * m_grid_gt * tk_grid), ak)
    return x_hat

x_hat = get_standard_form(ak, tk)

## 4. Find and implement $ G $ 

Once the signal is in form of 2, we need to identify how it is related to measurements y. 

*PEN AND PAPER: find the expression of matrix $G$ such that $ G \hat{x} = y$*

In [ ]:
def get_G(omega_ell):
    '''
    Compute G such that y=Gx
    
    :param omega_ell: vector of sampling frequencies.
    :return: matrix G 
    '''
    # TODO replace this with raw code
    G = build_G_fourier(omega_ell, M, TAU, interp_kernel)
    return G

m_limit = np.floor(M * TAU / 2.).astype(int)
omega_uniform = 2 * np.pi / TAU * np.arange(-m_limit, m_limit + 1)

tk_grid, omega_grid = np.meshgrid(tk, omega_uniform)
x_hat = np.dot(np.exp(-1j * omega_grid * tk_grid), ak)
print(x_hat)

G = get_G(omega_ell)

## generate noiseless signal
y_ell_test_real = np.dot(G, x_hat.real)
y_ell_test_imag = np.dot(G, x_hat.imag)
y_ell_test = y_ell_test_real + 1j * y_ell_test_imag

# TODO make sure that this assertion holds for certain signal types
#assert np.isclose(y_ell_samp, y_ell_test).all(), '{}'.format(y_ell_samp - y_ell_test)

plt.figure()
plt.plot(omega_continuous, np.real(y_ell_continuous), 'grey', label='continuous')
plt.plot(omega_ell, np.real(y_ell_test), 'g*', label='interpolated')
plt.plot(omega_ell, np.real(y_ell_samp), 'y*', label='samples')
plt.plot(omega_uniform, np.real(x_hat), 'r*', label='uniform')
plt.legend()
plt.show()


## 5. Solve optimization

Now we have all the ingredients to solve the optimization of the form: 

<center>
find $ \hat{x}, h $ 
</center>

<center>
such that $ || y - G \hat{x} ||_2 \leq \epsilon $
</center>

<center>
and $ \hat{x} * h = 0 $
</center>

*CODE: you do not have to implement this part, just inspect the obtained solution and make sure it is correct.*

In [ ]:
interp_kernel = 'dirichlet'
noise_level = np.max([1e-14, linalg.norm(noise)])
max_ini = 50  # maximum number of random initialisations
tk_recon, ak_recon, x_hat = \
    dirac_recon_irreg_fourier(y_ell, K, TAU, omega_ell, M,
                              noise_level, max_ini, interp_kernel=interp_kernel)
    
print(x_hat)

In [ ]:
# location estimation error
t_error = distance(tk_recon, tk)[0]

# plot reconstruction
plt.close()
fig = plt.figure(num=1, figsize=(5, 4), dpi=90)

subplt_height = 0.2
subplt_width = 0.87
subplt_left_corner = 0.115
# sub-figure 1
ax1 = plt.axes([subplt_left_corner, 0.71, subplt_width, subplt_height])
markerline311_1, stemlines311_1, baseline311_1 = ax1.stem(tk, ak, label='Original Diracs')
plt.setp(stemlines311_1, linewidth=1.5, color=[0, 0.447, 0.741])
plt.setp(markerline311_1, marker='^', linewidth=1.5, markersize=8, 
         markerfacecolor=[0, 0.447, 0.741], mec=[0, 0.447, 0.741])
plt.setp(baseline311_1, linewidth=0)

markerline311_2, stemlines311_2, baseline311_2 = \
    plt.stem(tk_recon, ak_recon, label='Estimated Diracs')
plt.setp(stemlines311_2, linewidth=1.5, color=[0.850, 0.325, 0.098])
plt.setp(markerline311_2, marker='*', linewidth=1.5, markersize=10,
         markerfacecolor=[0.850, 0.325, 0.098], mec=[0.850, 0.325, 0.098])
plt.setp(baseline311_2, linewidth=0)
ax1.yaxis.set_tick_params(labelsize=8.5)
plt.axhline(0, color='k')
plt.xlim([-TAU / 2, TAU / 2])
plt.ylim([1.18 * np.min(np.concatenate((ak, ak_recon, np.array(0)[np.newaxis]))),
          1.18 * np.max(np.concatenate((ak, ak_recon, np.array(0)[np.newaxis])))])
plt.xlabel(r'$t$', fontsize=12)
plt.ylabel(r'amplitudes', fontsize=12)
ax1.xaxis.set_label_coords(0.5, -0.21)
plt.legend(numpoints=1, loc=0, fontsize=9, framealpha=0.3,
           columnspacing=1.7, labelspacing=0.1)
t_error_pow = np.int(np.floor(np.log10(t_error)))

# sub-figure 2

G_conti_recon = build_G_fourier(omega_continuous, M, TAU, interp_kernel, tk_recon=tk_recon)
Xomegas_conti_recon = np.dot(G_conti_recon, Xomega_Uniform_ref)

ax2 = plt.axes([subplt_left_corner, 0.358, subplt_width, subplt_height])
line312_1 = ax2.plot(omega_ell, np.real(y_ell), label='Measurements')
plt.setp(line312_1, marker='.', linestyle='None', markersize=5, color=[0, 0.447, 0.741])

line312_2 = plt.plot(omega_continuous, np.real(y_ell_continuous), label='Ground Truth')
plt.setp(line312_2, linestyle='-', color=[0.850, 0.325, 0.098], linewidth=1)

line312_3 = plt.plot(omega_continuous, np.real(Xomegas_conti_recon), label='Reconstruction')
plt.setp(line312_3, linestyle='--', color=[0.466, 0.674, 0.188], linewidth=1.5)
plt.ylim([1.1 * np.min(np.concatenate((np.real(y_ell_continuous), np.real(y_ell)))),
          1.1 * np.max(np.concatenate((np.real(y_ell_continuous), np.real(y_ell))))])
ax2.yaxis.major.locator.set_params(nbins=7)
plt.ylabel(r'$\Re\left\{X(\omega)\right\}$', fontsize=13)
plt.legend(numpoints=1, loc=4, bbox_to_anchor=(1.013, 0.975), fontsize=9,
           handletextpad=.2, columnspacing=1.7, labelspacing=0.1, ncol=3)

## 6. Reconstruct original signal

Now that we have extracted the filter and $\hat{x}$, what would you do to find the signal's parameters ?

In [ ]:
fig = plt.figure(num=1, figsize=(5.5, 2.5), dpi=90)
ax1 = plt.axes([0.125, 0.59, 0.85, 0.31])

title = 'reconstructed vs. original signal'
plot_diracs(ax1, tk, ak, tk_recon, ak_recon, title)